# **Understanding the Heartbleed Bug**

## Introduction
The Heartbleed Bug is a serious vulnerability in the popular OpenSSL cryptographic software library. It allows stealing the information protected by the SSL/TLS encryption, which is used to secure the internet. This bug allows anyone on the internet to read the memory of systems protected by the vulnerable versions of OpenSSL.

## The Heartbeat Protocol
Before diving into the bug itself, it's crucial to understand the heartbeat protocol in TLS. A heartbeat message is usually sent to ensure that the other end is still connected and responsive. This message contains a payload, its length, and some random padding data.

#### **Heartbeat Message Format**
```plaintext
| Type (1 byte) | Payload Length (2 bytes) | Payload (variable) | Padding (variable) |
```


### Simulating a Heartbeat Request
Let's create a sample heartbeat request. The type is `0x01`, indicating a request, and the payload is `HelloRANDOM`.

In [ ]:
from avicenna_formalizations.heartbeat import heartbeat_string_to_hex
# Simulating a Heartbeat request
s = "\x01 5 Hello XY"
heartbeat_request = heartbeat_string_to_hex(s)
print("Heartbeat Request:", heartbeat_request)

## Simulating the Heartbleed Bug
Given the heartbeat request, if the payload length is set longer than the actual payload, a vulnerable server might send back more data than it should, revealing sensitive information from its memory.


In [ ]:
s = "\x01 5 Hello XY"
heartbeat_request = heartbeat_string_to_hex(s)
print("Heartbeat Request:", heartbeat_request)

In [ ]:
from avicenna_formalizations.heartbeat import heartbeat_response

response = heartbeat_response(heartbeat_request)
print("Heartbeat Response:", response)

In [ ]:
from avicenna_formalizations.heartbeat import hex_to_heartbeat_string

decoded_response = hex_to_heartbeat_string(response)
print("Decoded Response:", decoded_response)

Now we lets check the following heartbeat request:

In [ ]:
s = "\x01 100 Hello XY"
heartbeat_request = heartbeat_string_to_hex(s)
print("Heartbeat Request:", heartbeat_request)

In [ ]:
# Getting a response from the vulnerable server
response = heartbeat_response(heartbeat_request)
print("Heartbeat Response:", response)

In [ ]:
decoded_response = hex_to_heartbeat_string(response)
print("Decoded Response:", decoded_response)

In [ ]:
def test_heartbleed_vulnerability(request_str, response_hex):
    """Test if the Heartbleed bug occurred based on the request and response."""
    
    # Extract the specified payload length from the request string format
    # Given your format "\x01 8 Hello abc", we extract the number after the space and before the payload.
    specified_payload_length = int(request_str.split()[1])
    
    # Extract the actual payload from the request string (assuming space-separated format)
    actual_payload = request_str.split()[2][:specified_payload_length].encode('utf-8')
    
    # Convert the response hex string to bytes
    response_byte_data = bytes.fromhex(response_hex.replace(" ", ""))
    
    # Extract the payload (and potentially extra data) from the response
    response_payload_and_extra = response_byte_data[3:3+specified_payload_length+len(actual_payload)]

    # Compare the response payload to the request payload
    # If they're not the same or if there's extra data, then there's a potential Heartbleed vulnerability
    if response_payload_and_extra != actual_payload:
        return True  # Heartbleed bug might have occurred
    
    return False

In [ ]:
from avicenna.oracle import OracleResult
def oracle(inp):
    try:
        heartbeat_request_str = str(inp)
        hex_request = heartbeat_string_to_hex(heartbeat_request_str)
        response = heartbeat_response(hex_request)
        is_vulnerable = test_heartbleed_vulnerability(heartbeat_request_str, response)
        print("Is the server vulnerable to Heartbleed?", is_vulnerable)
    except OverflowError:
        return OracleResult.UNDEF
    return OracleResult.BUG if is_vulnerable else OracleResult.NO_BUG

In [ ]:
oracle("\x01 65535 Hello abc")

### Conclusion
The Heartbleed bug is a severe vulnerability that exposes sensitive data due to incorrect handling of heartbeat requests. It emphasizes the need for rigorous testing and scrutiny of cryptographic software and protocols.


In the case of the Heartbleed bug:

- A malicious client can send a heartbeat request with a payload of small size (let's say 1 byte) but specify a much larger payload length (e.g., 65535 bytes).
- A vulnerable server will then respond with a heartbeat response that contains the 1-byte payload and up to 65534 bytes of memory contents that follow it, which is the heart of the vulnerability. The padding is not the focal point here; the danger lies in the extra memory content returned as payload due to erroneous handling of the specified payload length.

In [ ]:
from isla.fuzzer import GrammarFuzzer
from avicenna_formalizations.heartbeat import grammar

g = GrammarFuzzer(grammar)

for _ in range(10):
    inp = g.fuzz()
    print(inp.encode())
    print(oracle(inp))

In [ ]:
from avicenna.avicenna import Avicenna
from avicenna_formalizations.heartbeat import initial_inputs, oracle

from isla.language import ISLaUnparser

avicenna = Avicenna(
    grammar=grammar,
    initial_inputs=initial_inputs,
    oracle=oracle,
    max_iterations=10,
)

diagnoses = avicenna.explain()

for diagnosis in diagnoses:
    print(ISLaUnparser(diagnosis[0]).unparse())

In [ ]:
for inp in avicenna.all_inputs:
    print(inp, inp.oracle)

In [ ]:
from isla.solver import ISLaSolver

constraint = """
exists <payload> container1 in start:
  exists <payload-length> length_field in start:
    (< (str.len container1) (str.to.int length_field))
"""
solver = ISLaSolver(grammar, constraint)

for inp in avicenna.all_inputs:
    bool_ = True if inp.oracle == OracleResult.BUG else False
    solver_result = solver.check(str(inp))
    if bool_ != solver_result:
        print(str(inp).encode())
    # print(str(inp).encode(), inp.oracle, solver.check(str(inp)))